In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm
from IPython.display import HTML
# import line_profiler as lp 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable, functional
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import wandb

import sys
sys.path.append('../src')
from model2D_v2 import *
from datagen import *
from analysis import *
from datagen2D import *

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU: ", device)
else:
    device = torch.device("cpu")
    print("Running on CPU")

torch.set_default_device(device)
# torch.cuda.synchronize()
# torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision("high")

/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on GPU:  cuda:0
Running on GPU:  cuda:0
Running on GPU:  cuda:0


/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mattis/anaconda3/envs/torch2/lib/python3.10/site-packages/umap/d

Running on GPU:  cuda:0


In [ ]:
# constant_2D_ratinaboxnew_1000_40_256.npy, gradual_2D_ratinabox_2000_10_64.npy, 
# gradual_2D_ratinabox_5000_25_64.npy, constant_2D_ratinaboxnew_400_300_64.npy
# constant_2D_ratinaboxnew_1000_40_256.npy

In [2]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="RNN-Navigation-Representations",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.0002,
    "architecture": "CfC_NCP_Low",
    "dataset": "gradual_2D_ratinabox_5000_25_64.npy",
    "dataset_generator": "datagen2D_OU",
    "low": True,
    "hidden_size": 256,
    "act_decay": 0.0,
    "weight_decay": 0.00,
    "act_decay_to_one": False,
    "h_bias": False,
    "input_bias": True,
    "rest_bias": None,
    "irnn": False,
    "activation": None,
    "clip_grad": 1.0,
    "optimizer": "SophiaG",
    }
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: mattis-dalsaetra. Use `wandb login --relogin` to force relogin


In [3]:
input_data = np.load("../datasets/" + config.dataset,allow_pickle=True)
config.epochs = input_data.shape[0]

In [5]:
# lr=0.0002,if_low=False,act_decay=0.0,weight_decay=0.01,act_decay_to_one=False,h_bias=False,input_bias=True,rest_bias=True,irnn=True,activation=True,clip_grad=True,optimizer="Sophia"

lr = config.learning_rate
input_size = 2
hidden_size = config.hidden_size


model = CfC_NCP(input_size,hidden_size,if_low=config.low,lr=lr)
model = torch.compile(model)

train = True
if train:
    _ = model.train_gradual_manual(input_data)
    model.plot_losses(average=30)
else:
    pass

Total/Task loss: 1.35648/1.35648:  32%|███▎      | 325/1000 [01:17<02:41,  4.18it/s]


KeyboardInterrupt: 

In [6]:
for i in range(len(model.task_losses)):
    wandb.log({"Task_loss": model.task_losses[i], "Total_loss": model.total_losses[i]})

In [7]:
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Task_loss,█▇▇▆▄▆▅▄▅▄▃▄▄▂▂▂▂▂▂▂▁▁▁▁▂▁▁▂▂▁▂▁▂▁▁▂▂▂▁▂
Total_loss,█▇▇▆▄▆▅▄▅▄▃▄▄▂▂▂▂▂▂▂▁▁▁▁▂▁▁▂▂▁▂▁▂▁▁▂▂▂▁▂
Task_loss,1.45056
Total_loss,1.45056
